# Data Science Assessment

### Prerequisite
The data source used is the <strong style="color: green">green</strong> taxi data for February 2016 from the following website: https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page (the files are stored in Parquet format).

In [ ]:
# Download the data source
!mkdir data
!wget -P ./data/ https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2016-02.parquet 

### Start of the Assessment

In [ ]:
# automatically reload imported packages
%load_ext autoreload
%autoreload 2

# import all the necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller # test for stationarity
import statsmodels.api as sm
import numpy as np
import pytz
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from itertools import product # cartesian product for gridsearch

# import libraries that I created that may be useful in the future (who knows)
from forecasting.plotting import datetime_line_plot, count_comparison_bar_plot, comparison_line_plot

# other setup
from IPython.core.interactiveshell import InteractiveShell
import warnings

#allow multiple notebook displays without printing or `display`
InteractiveShell.ast_node_interactivity = "all"
#view all the columns when displayed in a notebook
pd.set_option('display.max_columns', None)
#set matplotlib default style
plt.style.use('ggplot')
warnings.filterwarnings("ignore") # not recommended but for this assessment only

In [131]:
# read the data
trip_data = pd.read_parquet('./data/green_tripdata_2016-02.parquet')

#### 1. Report the number of rows and columns of the data.

In [ ]:
num_rows, num_columns = trip_data.shape
print(f"Number of rows: {num_rows}, Number of columns: {num_columns}")

#### Data Exploration

In any machine learning or statistical modelling, the most important phase (at least for me) is the data exploration. In this phase, the understanding of the data should be established and documents related to the data should be provided (like data dictionary).

Even if the objective of the forecasting modelling is the to forecast the number of trips per hour. Having an access to a data dictionary is important.

I've tried to look at the data dictionary of the taxi data and found the following table:

|Field Name| Description|
|----------|------------|
|VendorID | A code indicating the LPEP provider that provided the record. <br> 1= Creative Mobile Technologies, LLC; <br> 2= VeriFone Inc.|
lpep_pickup_datetime | The date and time when the meter was engaged.|
lpep_dropoff_datetime | The date and time when the meter was disengaged.|
Passenger_count | The number of passengers in the vehicle. This is a driver-entered value.|
Trip_distance | The elapsed Trip Distance in Miles reported by the taximeter.|
PULocationID | TLC Taxi Zone in which the taximeter was engaged|
DOLocationID | TLC Taxi Zone in which the taximeter was disengaged|
RateCodeID | The final rate code in effect at the end of the trip. <br> 1= Standard rate <br> 2=JFK <br> 3=Newark <br> 4=Nassau or Westchester <br> 5=Negotiated fare <br> 6=Group ride
Store_and_fwd_flag | This flag indicates whether the trip record was held in vehicle memory before sending to the vendor, aka “store and forward,” because the vehicle did not have a connection to the server. <br> Y= store and forward trip <br> N= not a store and forward trip|
Payment_type | A numeric code signifying how the passenger paid for the trip. <br> 1= Credit card <br> 2= Cash <br>3= No charge <br>4= Dispute <br>5= Unknown<br> 6= Voided trip |
Fare_amount | The time-and-distance fare calculated by the meter.|
Extra | Miscellaneous extras and surcharges. Currently, this only includes the $0.50 and $1 rush hour and overnight charges.|
MTA_tax | $0.50 MTA tax that is automatically triggered based on the metered rate in use.|
Improvement_surcharge | $0.30 improvement surcharge assessed on hailed trips at the flag drop. The improvement surcharge began being levied in 2015. |
Tip_amount | This field is automatically populated for credit card tips. Cash tips are not included.
Tolls_amount | Total amount of all tolls paid in trip. |
Total_amount | The total amount charged to passengers. Does not include cash tips. |
Trip_type | A code indicating whether the trip was a street-hail or a dispatch that is automatically assigned based on the metered rate in use but can be altered by the driver. <br>1= Street-hail <br>2= Dispatch

A few important points to note:
1. The distance is measured in miles. It might be easier to convert this to kilometers, as it's a metric unit and could provide a more intuitively precise measurement.
2. There's no timezone specified in the data dictionary, but since this data is from NYC, I can assume the timezone is EST/US Time.

Converting these two columns to their respective units/data types will help us draw more accurate insights from our analysis.

In [ ]:
trip_data.head()

In [ ]:
trip_data.info()

Two columns do not contain any data, and the trip_type has missing two data points.

<u>Normally, I am deleting the columns that do not contain any data. In ACSS(my current employer), we always make sure that this is well communicated to the clients, to let them be aware and for them to check if there are any errors in the data collection.</u>


In [ ]:
# To have an easier time dealing with time-series data, I always use DateTimeIndex
trip_data = trip_data.set_index('lpep_pickup_datetime') # can also use inplace=True
trip_data.head()

Making sure as well that we are in eastern timezone because the data is from NYC.

In [8]:
trip_data.index = trip_data.index.tz_localize(pytz.utc).tz_convert('US/Eastern')

In [ ]:
# checking the dtype
trip_data.index.dtype

Converting miles to kilometers:

In [ ]:
km_constant = 1.609344 # 1 mile  = 1.609344km

trip_data['trip_distance_km'] = trip_data['trip_distance'] * km_constant
trip_data[['trip_distance_km', 'trip_distance']].head()

*<p style="font-size:11px; color:orange">For a non forecasting models, after reviewing the data dictionary, and consulting some concerns with the client, I always perform the univariate, bivariate, and multivariate analysis to the variables of interest.</p>*

In [ ]:
# this code has been stored in the folder ./forecasting/plotting.py
datetime_line_plot(trip_data,
                   'trip_distance_km',
                   plot_title_name = 'Trip Distance (KM) by Time of Day', 
                   plot_xlabel_name= 'Date and Time', 
                   plot_ylabel_name= 'Trip Distance in Kilometers')
print("Figure 1: Trip Distance (KM) by Time of Day")

Without downsampling the data (averaging the trip distance per hour), we can see outliers in the first week of February. There are also longer trips in the middle of the month, which could mean that people tend to have longer trips on salary day/pay day (Assuming the payday are every 15th and 30th of the month), as stated in the NYC labor law: https://dol.ny.gov/frequency-pay, which mandates that clerical and other workers are paid at least twice per month.

Plotting the average hourly trip:

In [ ]:
datetime_line_plot(trip_data,
                   'trip_distance_km',
                   freq='h',
                   plot_title_name = 'Average Hourly Trip Distance (KM) by Time of Day', 
                   plot_xlabel_name= 'Date and Time', 
                   plot_ylabel_name= 'Trip Distance in Kilometers')
print("Figure 2: Average Hourly Trip Distance (KM) by Time of Day")

We can observe that our trip is somehow stationary, which is what we need to consider when forecasting using ARIMA models. (But this will not be modeled, since the number of trips will be forecasted not the Hourly Distance)

The created `datatime_line_plot` function has a `make_interactive` parameter to look at the line plot in more detailed.

In [ ]:
datetime_line_plot(trip_data,
                   'trip_distance_km',
                   freq='h',
                   plot_title_name = 'Average Hourly Trip Distance (KM) by Time of Day', 
                   plot_xlabel_name= 'Date and Time', 
                   plot_ylabel_name= 'Trip Distance in Kilometers',
                   make_interactive=True)
print("Figure 3: Average Hourly Trip Distance (KM) by Time of Day Interactive Plot")


The following assumptions formulated are: 

1. Many people may be taking taxis for longer late night trips, could indicate that many people are using taxis for weekend getaways or business trips during the week to go to airport from their homes.
2. There are shorter daytime trips, this suggests that taxis are often used for commuting to work, likely for people who live relatively close to their workplaces.
3. This may reflect that people tend to stay closer to home, spending time with family rather than traveling long distances (or visit a friend)

#### 3. Popular pickup locations on weekdays vs weekends

In [ ]:
# extract first the day of week first
trip_data['day_of_week'] = trip_data.index.day_name()
trip_data['day_of_week'].value_counts()

In [15]:
WEEK_ENDS = ['Saturday', 'Sunday']

In [ ]:
# filter the week days and the week ends
week_ends_mask = trip_data['day_of_week'].isin(WEEK_ENDS)
week_end_trips_data = trip_data[week_ends_mask].copy()
week_day_trips_data = trip_data[~week_ends_mask].copy()

# to check the filtered week end and week day
week_end_trips_data['day_of_week'].unique()
week_day_trips_data['day_of_week'].unique()

In [ ]:
count_comparison_bar_plot(week_day_trips_data, week_end_trips_data,
                          column_name='PULocationID', df1_name='Weekdays', df2_name='Weekends', 
                          plot_title_name = 'Pickup Locations Count: Weekdays vs Weekends',
                          plot_xlabel_name = "Rank Number",
                          plot_ylabel_name = 'Trip Count')
print("Figure 4: Pickup Locations Count: Weekdays vs Weekends")

The above plot shows the top 10 most frequent pick up locations. The top 1 is the location ID 75 on weekdays, and location ID 41 for week ends. 
However, the difference between the number of counts of weekdays and weekends is large, this is because weekday has 5 days, and weekend has 2. 
To solve this, we can compute the ratio, then rank the location ID. I can achieve this by adding a paramter `make_ratio = True`

In [ ]:
count_comparison_bar_plot(week_day_trips_data, week_end_trips_data,
                          make_ratio=True,
                          column_name='PULocationID', df1_name='Weekdays', df2_name='Weekends', 
                          plot_title_name = 'Pickup Locations Proportion: Weekdays vs Weekends',
                          plot_xlabel_name = "Rank Number",
                          plot_ylabel_name = 'Trip Ratio')
print("Figure 5: Pickup Locations Proportion: Weekdays vs Weekends")


With this, we can say that the top 10 frequent location does not vary significantly and the ranking of the ratio is similar to the count.

In conclusion, designating more taxis to location IDs 75, 41, 74, 7, 166, ..., and 97 on weekdays, and to location IDs 41, 255, 7, 74, 82, ..., and 97 on weekends, could optimize fleet utilization. The higher demand in these areas indicates that deploying more taxis there would likely capture a greater share of trips, leading to increased revenue. By aligning taxi ranking distribution with these demand patterns, the company can more effectively serve passengers and maximize earnings.

The idea of fleet utilization has comes from here:
- https://www.way.com/blog/fleet-use-meaning/
- https://www.tourmo.ai/resources/learn/how-to-improve-fleet-utilization-in-3-steps-tm/

#### 4. Build a model to forecast the number of trips per hour (ARIMA)


In this model, I will try to implement ARIMA model, and one requirement that we need to satify is the stationarity of the data.

We have three ways to measure the stationarity of the data. These are the following:
1. Visual inspection using time-series plot. If we see that we have constant mean and variance, and no seasonality, then we have stationary data. If the visual is hard to visualize, then we go to numbers 2 and 3 below.
2. Using global tests. In this case, we measure the mean and std of the whole data, then we compute the mean of only a portion of data (For example: Jan to Feb, and the global mean is from Jan to Dec). If we have approimately similar mean and standard deviation, with no seasonality, then we can say that our data is stationary.
3. Using augmented dickey-fuller test. 

This notebook only shows 1 and 3:

In [ ]:
datetime_line_plot(trip_data,
                   'trip_distance',
                   freq='h',
                   agg_operation='count',
                   plot_title_name = 'Average Hourly Trip Count by Time of Day', 
                   plot_xlabel_name= 'Date and Time',
                   plot_ylabel_name= 'Number of Trips',
                   make_interactive=True)
print("Figure 6: Average Hourly Trip Count by Time of Day")

Figure 6 shows that the number of trips per hour peaks in the early evening / late afternoon, particularly between 16:00 and 17:00, which aligns with the end of typical working hours. In contrast, the lowest trip counts occur between 20:00 and 01:00, with fewer than a thousand trips during these hours. Based on Figure 3, which displays the average hourly trip distance by time of day, it appears that trips during these low-count hours tend to be longer. This may be due to business-related travel on weekdays or vacation trips on weekends, possibly including trips to the airport. 

Also, since there are peaks that are significantly larger than the other, I think this is non-stationary. To confirm this hypothesis, the augmented dickey-fuller test has been provided:

In [ ]:
# get the hourly count
hourly_count_trip_data = trip_data.resample('h').size()
hourly_count_trip_data

Confirming the initial assumption of non-stationarity, I calculate the augmented dickey-fuller

In [ ]:
adf_result = adfuller(hourly_count_trip_data)
print(f"Test Statistics = {adf_result[0]}\nP-value = {adf_result[1]}")

The augmented dickey-fuller test above shows that our data is non-stationary, since we have higher p-value (In this case, I will use a 0.05 p-value to reject the null hypothesis). To make the data stationary, we can use differencing:

In [22]:
def get_differencing(data, order=1):
    """
        I think this can be done using recursion
    """
    data = np.array(data)
    differenced_data = [current_point - data[index-1] for index, current_point in enumerate(data) if index != 0]
    
    if order == 1:
        return differenced_data
    elif order > 1:
        order=-1
        return get_differencing(differenced_data, order=order)
    else:
        raise ValueError("Order is invalid")


I coded the above differencing, however, this can be easily achieved in pandas using `diff()` method. (adding two `diff()` methods would result to two levels  of differencing `diff().diff()`.)

In [ ]:
diffed_hourly_count_trip_data = hourly_count_trip_data.diff()
diffed_hourly_count_trip_data.head()

Computing the ADF again:

In [ ]:
adf_result = adfuller(diffed_hourly_count_trip_data.fillna(method='bfill')) # I can dropna as well, it doesn't impact the result since it's only one observation
print(f"Test Statistics = {adf_result[0]}\nP-value = {adf_result[1]}")

The result now shows stationarity.

In [ ]:
datetime_line_plot(diffed_hourly_count_trip_data.to_frame().rename(columns={0: "count"}),
                   "count",
                   plot_title_name = 'Differenced Average Hourly Trip Count by Time of Day', 
                   plot_xlabel_name= 'Date and Time',
                   plot_ylabel_name= 'Differenced Number of Trips',
                   make_interactive=True)
print("Figure 7. Differenced Average Hourly Trip Count by Time of Day")

The above process shows how we convert the data to stationary, this is important since I can now specify that I only need one differencing to convert the data to stationary, which is useful in parameterizing the ARIMA model. Since I have now the first parameter, I can now look at the ACF and PACF plot for the Moving Average part and the Autoregressive part of the ARIMA model.

**ACF AND PACF Plot**

In [ ]:
# from statsmodels.api import graphics
sm.graphics.tsa.plot_acf(diffed_hourly_count_trip_data.dropna())
plt.show()

Looking at the ACF plot, we can estimate the order for the moving average part. In this case, since the lags two does show a significant positive correlation and the next lag (lag 3) is not, we can set the moving average part order to 2.

In [ ]:
sm.graphics.tsa.plot_pacf(diffed_hourly_count_trip_data.dropna())
plt.show()

For the autoregressive part, I looked at the PACF, similar with the moving average part of the ARIMA model, I can set the order to 2.

Finally, I can set the ARIMA order of (2, 1, 2).

**Looking at the trend, seasonality, and residual.**

In [ ]:
decomposition = sm.tsa.seasonal_decompose(hourly_count_trip_data, model='additive')
fig = decomposition.plot()
plt.show()
print("Figure 8. Observed, Trend, Seasonal, and Residual Plot of the Number of Hourly Trip")

Since we have observed that there's a seasonality, I think the model that we can use here is a Seasonal ARIMA. However, due to the time constraints, I cannot observe the seasonal order of the Seasonal ARIMA. However, since we only have February data, I think I can safely avoid using seasonality in this modelling.

Now, for the actual modelling.

In [111]:
model = sm.tsa.arima.ARIMA(hourly_count_trip_data, order=(2,1,2))
fitted_model = model.fit()

In [ ]:
num_days = 12

start_index = pd.to_datetime('2016-02-12 10:00:00', utc=True).tz_convert('US/Eastern')
end_index = start_index + pd.DateOffset(days=num_days)
print(f"Forecast from {str(start_index)} to {end_index}")

In [113]:
pred = fitted_model.get_prediction(start=start_index, end=end_index, dynamic=False)
pred_ci = pred.conf_int()

In [ ]:
groud_truth_hourly_trip = hourly_count_trip_data[start_index:end_index].to_frame().rename(columns={0: "forecast"})
predicted_values = pred.predicted_mean.to_frame().rename(columns={"predicted_mean": "forecast"})
comparison_line_plot(groud_truth_hourly_trip,
                     predicted_values,
                     column_name="forecast",
                     df1_name="True Values",
                     df2_name="Predicted Values",
                     plot_title_name = '12 Day Forecast: True Values vs Predicted Values', 
                     plot_xlabel_name= 'Date and Time',
                     plot_ylabel_name= 'Forecast')


Looking at the prediction vs the true values. We see that model that I created can follow the trend of the data. Therefore, I think this is already a good (okay) model. However, I need to look at the plot_diagnostics and coefficients to see if the statistical assumptions have been met.


In [ ]:
fitted_model.summary()

Looking at the summary although the predicted values are following the trend of the true values. Our statistical analysis says that our model is not good. The following are the reason:
1. The ar.L1 and ar.L2 is the p parameter of our ARIMA model (the autoregressive part), since the P>|z| is not small, this could mean that the this lowly impacts the time-series model prediction. We need to think of a better way find the best value of the p (autoregressive part) parameter of the model or properly transform our data.
2. The jarque bera p-value is low, which indicates we reject the idea that the residual is normally distributed. If we implement this model, we wouldn't get a valid confidence interval, coefficient estimates, and p-value.
3. Ljung-Box (L1) is 0, which means that our residuals are not independent, which is also an important assumption.

In this case, I think it is safe to asume that ARIMA cannot satify all the assumptions(the order that I inferred earlier may not be the optimal order). Other models, data transformations, and order should be considered as well. For now, I will evaluate the developed model.

#### 5. Evaluate your forecast model and outcomes using any measures you think are appropriate.

In [ ]:
fitted_model.plot_diagnostics(figsize=(15, 12))
plt.tight_layout()
plt.show()

In [ ]:

baseline = groud_truth_hourly_trip.mean()
print(f'The baseline {round(baseline, 2)}')

rmse = np.sqrt(((groud_truth_hourly_trip - predicted_values) ** 2).mean())
print(f'The Mean Squared Error of our forecasts is {round(rmse, 2)}')

mae = abs(groud_truth_hourly_trip - predicted_values).mean()
print(f'The Mean Squared Error of our forecasts is {round(mae, 2)}')

r2 = r2_score(groud_truth_hourly_trip, predicted_values)
print(f'R Squared of our forecasts is {round(r2, 2)}')


Although some statistical tests are not satisfied. The error of the prediction of the model is way lower than the baseline, which is the average count of the hourly trip, which is good. To support this, the R2 is also provided with 0.94, which means 94 of the variance of the variance can be explained by the model.

Future improvements:
1. Perform out-of-sample tests
2. Try other models and other transformations to satisfy the statistical assumptions of the model.
3. Find the most optimal order of ARIMA model.

In [141]:
# Optimizing the parameter using grid search

# warnings.filterwarnings("ignore") # specify to ignore warning messages

# p = range(0, 2)
# d = range(1, 2)
# q = range(0, 2)
# for order in product(p, d, q):
#     for seasonal_order in product(range(0, 5), range(0, 2), range(0, 5)):
#         mod = sm.tsa.statespace.SARIMAX(hourly_count_trip_data,
#                                 order=order,
#                                 seasonal_order=(*seasonal_order, 12))
#         result = mod.fit(disp=False)
#         print(order, seasonal_order, 12)
#         print(result.aic)
